In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

df = pd.read_csv('./full_lyrics_polarity_dataset.csv')
df

,Unnamed: 0,artist,title,year,lyrics,neg_sentiment,neu_sentiment,pos_sentiment,com_sentiment,polarity,textblob_pol
0,0,frankie laine,i believe,1950,I Believe Lyrics\nI believe for every drop of ...,0.089,0.879,0.032,-0.7506,0.0,1.0
1,1,johnnie ray,cry,1950,Cry Lyrics\nOoh-wah\nOoh-wah\nOoh-wah\n\n\nIf ...,0.110,0.772,0.118,0.2732,1.0,1.0
2,2,p?rez prado,patricia,1950,Patricia LyricsKiss her and your lips will alw...,0.029,0.747,0.224,0.9807,1.0,1.0
3,3,lefty frizzell,if you've got the money i've got the time,1950,If You’ve Got The Money I’ve Got The Time Lyri...,0.038,0.920,0.042,0.0112,0.0,1.0
4,4,lefty frizzell,i want to be with you always,1950,I Want To Be With You Always Lyrics\nI lose my...,0.074,0.702,0.224,0.9835,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
21739,21739,mack 10,10 million ways,2019,10 Million Ways Lyrics\nCause if you fuck with...,0.133,0.807,0.059,-0.9936,0.0,1.0
21740,21740,m.o.p.,ante up (robbin hoodz theory),2019,Ante Up \nMotherfucker!\nYeah!23Embed,0.180,0.730,0.090,-0.9972,0.0,0.0
21741,21741,nine,whutcha want?,2019,Whutcha Want? Lyrics\nI gets banned if I do ge...,0.101,0.782,0.117,-0.7261,0.0,1.0
21742,21742,will smith,switch,2019,"Switch Lyrics\nYo mic check, mic check, yeah h...",0.041,0.888,0.071,0.9630,1.0,1.0


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['lyrics'], df['polarity'], test_size = 0.25, random_state = 32)
X_train.shape

(16308,)

In [4]:
from transformers import BertTokenizer

# BERT에 맞는 Tag 달아주기
train_lyrics = []
test_lyrics = []

for i in X_train:
  one = []
  start = "[CLS] " + str(i)
  split_lyrics = " [SEP]".join(start.split('\n'))
  split_lyrics += " [SEP]"
  one.append(split_lyrics)
  train_lyrics.append(one)

# train_lyrics_df = pd.DataFrame(train_lyrics, columns=["lyrics_tag"])

for i in X_test:
  one = []
  start = "[CLS] " + str(i)
  split_lyrics = " [SEP]".join(start.split('\n'))
  split_lyrics += " [SEP]"
  one.append(split_lyrics)
  test_lyrics.append(one)

# test_lyrics_df = pd.DataFrame(test_lyrics, columns=["lyrics_tag"])
# train_lyrics_df

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)
'''bert-base-multilingual-cased''' 
'''bert-base-uncased''' 
tokenized_data = []

# 2차원 리스트로 태그를 달아주었으므로 2중 for 문 사용해서 가사에 접근
for each_lyrics in train_lyrics:
  for j in each_lyrics:
    tokens = tokenizer.tokenize(j)
    tokenized_data.append(tokens)


tokenized_test_data = []
for each_lyrics in test_lyrics:
  for j in each_lyrics:
    tokens = tokenizer.tokenize(j)
    tokenized_test_data.append(tokens)

# tokenized_test_data_temp = pd.DataFrame(tokenized_test_data)

In [6]:
y_train_list = y_train.to_list()
y_test_list = y_test.to_list()

In [8]:
pip install tensorflow_datasets

  Using cached tensorflow_datasets-4.6.0-py3-none-any.whl (4.3 MB)
  Using cached promise-2.3.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached etils-0.7.1-py3-none-any.whl (124 kB)
  Using cached tensorflow_metadata-1.10.0-py3-none-any.whl (50 kB)
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached importlib_resources-5.9.0-py3-none-any.whl (33 kB)
     -------------------------------------- 211.7/211.7 kB 4.3 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=879e097d8be35c6603b60efbd459ab45f78bdfa5c32d0f5e559033551749f4a4
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\54\4e\28\3ed0e1c8a752867445bab994d2340724928aa3ab059c57c8db
Successfully built promise
Note: you may need to restart the kernel to use updated packages.


In [12]:
import tensorflow_datasets as tfds
import tensorflow as tf
import torch

def convert_example_to_feature(review):
  return tokenizer.encode_plus(review,
                add_special_tokens = False, # add [CLS], [SEP]
                max_length = 256, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens
                return_attention_mask = True, # add attention mask to not focus on pad tokens
              )
  
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(data):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  target_list = []
  
  for DATA_COL, LABEL_COL in data.to_numpy():
    bert_input = convert_example_to_feature(DATA_COL)
    input_ids_list.append(bert_input['input_ids'])
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    target_list.append([LABEL_COL])
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, target_list)).map(map_example_to_dict)

In [13]:
train = pd.DataFrame({'DATA_COL' : tokenized_data, 'LABEL_COL' : y_train_list})
test = pd.DataFrame({'DATA_COL' : tokenized_test_data, 'LABEL_COL' : y_test_list})

In [14]:
# train dataset
train_encoded = encode_examples(train).shuffle(100).batch(16)
# test dataset
test_encoded = encode_examples(test).batch(16)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [15]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf
# recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5
# we will do just 1 epoch, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 3
# model initialization
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [17]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

model.fit(train_encoded, epochs=number_of_epochs, validation_data=test_encoded)

Epoch 1/3
   1/1020 [..............................] - ETA: 7:55:19 - loss: 0.6731 - accuracy: 0.5625

KeyboardInterrupt: 

In [ ]:
test_sentence = '''
Jenna, time to pee on a stick
Wh-?
I don't wanna know
Come one honey, you've waited long enough
Get to it and do it!
Wh-?
Okay, girls!
Enough!
You know what we mean
I hope you drank enough this morning
Come on, sweetie, it's better to know
We'll be right here with you, so
It's no or it's yes
But either way you gotta take the test
A squat and a squeeze
A prayer and a please
It's nothing
A stick and a line
Just one of 'em, if i'm lucky
A pot and a piss
Here we go sis
And we'll keep our focus on
The negative!
Read the instructions
Se puede saber la duración de la-
English!
Do not insert the test stick into your vagina
Wow
Thank you, Dawn
How'd I ever get myself in this mess?
One drunk night in that stupid red dress
Oh, I love that red dress!
The way it sparkles
It looks like an ice skating outfit
Stay with us, Dawn!
Maybe his machinery is broken somehow
What if his boys don't swim?
I mean, wow!
Miraculous luck!
Yeah, miraculous!
To get away with an unprotected fu-
Funny how one night can ruin you're whole life
Don't go there yet
We don't know what the test says
I'm already panicked!
Just calm down, Godammit!
Maybe it'll all be fine
Maybe there'll be just one line
Come on, negative!
I thought you don't sleep much with you husband much anymore?
Well, I - he got me drunk
I do stupid things when I drink
Like sleep with my husband
Honey, we've all made that mistake
Focus on the negative!
This will all be alright
It was only one night
Someone send me a sign
One line
One line
That means the test is ready
This is it
Shit!
'''

predict_input = tokenizer.encode(test_sentence,

truncation=True,

padding=True,

return_tensors="tf")
tf_output = model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive'] #(0:negative, 1:positive)
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]],": {:.2f}%".format(abs(tf_output.take(label[0])) * 100))